In [1]:

%pip install elasticsearch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from docx import Document

def clean_word_remove_footer(input_path, output_path):
    # Mở tài liệu Word
    doc = Document(input_path)
    
    for section in doc.sections:
        # Xóa nội dung trong footer
        for paragraph in section.footer.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
        
        # Xóa nội dung trong header (nếu cần)
        for paragraph in section.header.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
    
    # Lưu tài liệu đã làm sạch
    doc.save(output_path)
    print(f"Footer và Header đã được làm trống. Tệp mới lưu tại: {output_path}")

# Sử dụng
clean_word_remove_footer("data\QUỐC HỘI.docx", "52.2014.QH13_clean.docx")


Footer và Header đã được làm trống. Tệp mới lưu tại: 52.2014.QH13_clean.docx


In [5]:
from docx import Document

def clean_word_based_on_keywords(input_path, output_path, keywords_to_remove_start, keywords_to_remove_end):
    """
    Xóa các phần đầu và cuối tài liệu dựa trên từ khóa xác định.
    
    input_path: Đường dẫn file gốc (.docx)
    output_path: Đường dẫn file kết quả (.docx)
    keywords_to_remove_start: Danh sách từ khóa để xác định phần đầu không liên quan.
    keywords_to_remove_end: Danh sách từ khóa để xác định phần cuối không liên quan.
    """
    doc = Document(input_path)
    paragraphs = doc.paragraphs
    new_doc = Document()

    # Tìm và loại bỏ các phần đầu không liên quan
    start_index = 0
    for i, paragraph in enumerate(paragraphs):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_start):
            start_index = i + 1  # Bắt đầu từ đoạn sau phần đầu không liên quan
        else:
            break

    # Tìm và loại bỏ các phần cuối không liên quan
    end_index = len(paragraphs)
    for i, paragraph in enumerate(reversed(paragraphs)):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_end):
            end_index = len(paragraphs) - i - 1  # Dừng lại ở đoạn trước phần cuối không liên quan
        else:
            break

    # Thêm các đoạn văn còn lại vào tài liệu mới
    for i, paragraph in enumerate(paragraphs[start_index:end_index]):
        new_doc.add_paragraph(paragraph.text)

    # Lưu tài liệu đã làm sạch
    new_doc.save(output_path)
    print(f"Đã làm sạch tài liệu. Tệp lưu tại: {output_path}")

# Sử dụng:
keywords_to_remove_start = ["Cộng hòa xã hội chủ nghĩa Việt Nam", "Quốc hội", "Chủ tịch"]
keywords_to_remove_end = ["Chữ ký", "Phê duyệt", "Chủ tịch"]

clean_word_based_on_keywords(
    "data\QUỐC HỘI.docx",
    "52.2014.QH13_clean.docx",
    keywords_to_remove_start=keywords_to_remove_start,
    keywords_to_remove_end=keywords_to_remove_end
)

Đã làm sạch tài liệu. Tệp lưu tại: 52.2014.QH13_clean.docx


In [6]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip show python-docx

Name: python-docx
Version: 1.1.2
Summary: Create, read, and update Microsoft Word .docx files.
Home-page: https://github.com/python-openxml/python-docx
Author: 
Author-email: Steve Canny <stcanny@gmail.com>
License: MIT
Location: c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: lxml, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
from docx import Document
def read_docx(file_path):
    # Đọc nội dung từ file .docx
    doc = Document(file_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

In [9]:
def roman_to_int(roman):
    roman_numerals = {
        "I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "VI": 6, "VII": 7, "VIII": 8, 
        "IX": 9, "X": 10
    }
    return roman_numerals.get(roman.upper(), None)  # Chuyển chữ hoa và trả về số nếu có

def normalize_chapter_number(chapter_number):
    # Nếu chương là số La Mã, chuyển sang số hệ 10
    try:
        # Nếu là số bình thường (hệ thập phân), không làm gì
        int(chapter_number)
        return chapter_number
    except ValueError:
        # Nếu là số La Mã, chuyển nó thành số hệ 10
        return roman_to_int(chapter_number)  # Trả về số hệ 10 nếu có

In [10]:
import re

def chunk_legal_text(text):
    chunks = []
    current_metadata = {"chuong": None, "muc": None, "dieu": None}

    # Tách văn bản thành từng dòng để dễ xử lý
    lines = text.splitlines()

    i = 0
    while i < len(lines):
        line = lines[i]
        # Bỏ qua các dòng trống
        if not line.strip():
            i += 1
            continue

        # Nhận diện CHƯƠNG
        if re.match(r"^CHƯƠNG ", line):
            match = re.match(r"^CHƯƠNG\s*(\S+)(.*)", line.strip())
            if match:
                chapter_number = match.group(1)
                normalized_chapter_number = normalize_chapter_number(chapter_number)
                current_metadata["chuong"] = f"chương {normalized_chapter_number}"
            
            current_metadata["muc"] = None  # Reset mục

            #Lấy tên chương trên dòng tiếp theo
            i += 1
            if i < len(lines) and lines[i].strip():
                current_metadata["chuong"] += f" {lines[i].strip()}"
            
        
        # Nhận diện Mục
        elif re.match(r"^Mục \d+", line):
            current_metadata["muc"] = line.strip()
            # Lấy tiêu đề của mục (dòng tiếp theo nếu có)
            i += 1
            if i < len(lines) and lines[i].strip():
                current_metadata["muc"] += f" {lines[i].strip()}"
        
        # Nhận diện Điều
        elif re.match(r"^Điều \d+\.", line):
            match_ = re.match(r"^(Điều \d+)\.\s*(.*)", line)
            if match_:
                current_metadata["dieu"] = match_.group(1)  # Lưu "Điều X."
                title_of_dieu = match_.group(2).strip()  # Lưu tên điều (nếu có)
            content_lines = []
            i += 1

            while i < len(lines):
                next_line = lines[i]
                if re.match(r"^(CHƯƠNG |Mục \d+|Điều \d+\.|\\d+\. )", next_line):
                    break
                content_lines.append(next_line.strip())
                i += 1

            # Phân loại điều có một khoản hoặc nhiều khoản
            if content_lines and any(re.match(r"^\d+\. ", line) for line in content_lines):
                # Điều có nhiều khoản
                current_chunk = ""  # Khởi tạo current_chunk là chuỗi rỗng thay vì None
                for idx, content_line in enumerate(content_lines):
                    if re.match(r"^\d+\. ", content_line):  # Bắt đầu khoản mới
                        # Gắn tên điều vào khoản đầu tiên
                        if idx == 0 and title_of_dieu:
                            content_line = f"{title_of_dieu} {content_line}"

                        # Tìm reference trong nội dung khoản
                        references = re.findall(r"Điều \d+", content_line)
                        # Thêm khoản chính vào chunk
                        if current_chunk:
                            chunks.append({
                                "chuong": current_metadata["chuong"],
                                "muc": current_metadata["muc"],
                                "dieu": current_metadata["dieu"],
                                "content": current_chunk.strip(),
                                "reference": references or None,
                            })
                        current_chunk = content_line.strip()  # Bắt đầu khoản mới
                    else:
                        # Xử lý khoản con (a), b), c)...)
                        current_chunk += " " + content_line.strip()  # Gộp vào cùng khoản

                # Nếu có khoản cuối không tách riêng, thêm vào chunks
                if current_chunk:
                    references = re.findall(r"Điều \d+", current_chunk)
                    chunks.append({
                        "chuong": current_metadata["chuong"],
                        "muc": current_metadata["muc"],
                        "dieu": current_metadata["dieu"],
                        "content": current_chunk.strip(),
                        "reference": references or None,
                    })
            else:
                # Điều chỉ có một khoản (nội dung không bắt đầu bằng số)
                body_of_dieu = " ".join(content_lines).strip()
                references = re.findall(r"Điều \d+", body_of_dieu)
                chunks.append({
                    "chuong": current_metadata["chuong"],
                    "muc": current_metadata["muc"],
                    "dieu": current_metadata["dieu"],
                    "content": f"{title_of_dieu} {body_of_dieu}".strip(),
                    "reference": references or None,
                })


        else: 
            i += 1

    return chunks


text = read_docx("data/52.2014.QH13_clean.docx")
text = text.replace("\xa0", " ")
chunks = chunk_legal_text(text)

for chunk in chunks:
    print(chunk)
    print("-" * 40)

{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 1', 'content': 'Phạm vi điều chỉnh Luật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách ứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước và xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình.', 'reference': None}
----------------------------------------
{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 2', 'content': 'Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình 1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.', 'reference': None}
----------------------------------------
{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 2', 'content': '2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người theo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người không có tín ngưỡng, giữa công dân Việt Nam với người nước ngoài được tôn trọng và được pháp 

In [11]:
from elasticsearch import Elasticsearch

# Kết nối Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Tên chỉ mục cần xóa
INDEX_NAME = "legal_documents"

# Kiểm tra và xóa chỉ mục
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"Chỉ mục '{INDEX_NAME}' đã bị xóa thành công.")
else:
    print(f"Chỉ mục '{INDEX_NAME}' không tồn tại.")


Chỉ mục 'legal_documents' không tồn tại.


In [2]:
%pip install -qU langchain-community langchain-elasticsearch
%pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from typing import Any, Dict, Iterable

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from langchain_community.embeddings import DeterministicFakeEmbedding
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_elasticsearch import ElasticsearchRetriever

In [4]:
es_url = "http://localhost:9200"
es_client = Elasticsearch(hosts=[es_url])
es_client.info()

ObjectApiResponse({'name': 'DESKTOP-MKQMC6P', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'xS9Mo8nRQ4Gcaq4QY6XO6g', 'version': {'number': '8.16.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'ffe992aa682c1968b5df375b5095b3a21f122bf3', 'build_date': '2024-11-19T16:00:31.793213192Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="bkai-foundation-models/vietnamese-bi-encoder")

In [12]:
index_name = "legal_document"

In [13]:
def create_index(
    es_client: Elasticsearch,
    index_name: str,
):
    es_client.indices.create(
        index=index_name,
        mappings={
            "properties": {
                "chuong": {"type": "text"},
                "muc": {"type": "text"},
                "dieu": {"type": "text"},
                "content": {"type": "text"},
                "reference": {"type": "text"},
                "embedding": {"type": "dense_vector"}
            }
        },
    )


def index_data(
    es_client: Elasticsearch,
    index_name: str,
    embeddings: Embeddings,
    data: Iterable[dict],
    refresh: bool = True,
) -> None:
        
    create_index(es_client, index_name)

    # Trích xuất nội dung để tạo embedding
    content_list = [item["content"] for item in data]
    vectors = embeddings.embed_documents(content_list)

    requests = [
        {
            "_op_type": "index",
            "_index": index_name,
            "_id": i,
            "chuong": item["chuong"],
            "muc": item.get("muc"),
            "dieu": item["dieu"],
            "content": item["content"],
            "reference": item.get("reference"),
            "embedding": vector,
        }
         for i, (item, vector) in enumerate(zip(data, vectors))
    ]

    bulk(es_client, requests)

    if refresh:
        es_client.indices.refresh(index=index_name)

    return len(requests)

In [14]:
index_data(es_client, index_name, embeddings, chunks)

338